In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

Number_level = 10;
#Transition_matrix = [0.86 0.14 0.0 0.0; 0.0 0.83 0.17 0.0;0.0 0.0 0.89 0.11;1.0 0.0 0.0 0.0];
fullname = "tm10";
# generate_transition_matrix(Number_level, fullname) 
Transition_matrix = load(fullname*".jld2","transition_matrix");
failure_penalty = -1000.0;
maintenance_penalty = -200.0;
setup_cost = -800.0;
normal_operation = 0.0;

In [6]:
function reward(s, a,s1=10, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
       if s==Number_level
            r=failure_penalty  + normal_operation  +  setup_cost/NumberUnits  ;
        return r
       end
        r=0
     if a==1
        if s1>=s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            return 0.0
        end
        end
#     if a==2
#          if s1>=s
#             crd = Categorical(Transition_matrix[s,:]);
#             s = rand(crd);
#             r = normal_operation +  setup_cost/NumberUnits;
#          return r
#         else
#             return 0.0
#         end
#         end
    if a==2
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty 
         crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r+= normal_operation  +  setup_cost/NumberUnits  ;
#            r*=(s1/10);
        return r
    end
    end

reward (generic function with 3 methods)

In [10]:
T=zeros(Number_level,Number_level,3);


In [11]:
((1:10)./10)

0.1:0.1:1.0

In [12]:
T[:,:,1].=Transition_matrix;

In [13]:
T[:,:,2].=Transition_matrix;

In [14]:
for j in 1:10
        T[j,:,2].=Transition_matrix[1,:];
        end
# for j in 1:10
#         T[j,:,3].=0.0
#     T[j,1,3]=1.0
#         end

In [15]:
T[10,:,1].=Transition_matrix[1,:]
T[10,:,2].=Transition_matrix[1,:]

10-element view(::Array{Float64, 3}, 10, :, 2) with eltype Float64:
 0.2695279572108137
 0.16860741623354286
 0.1306613209439592
 0.09857168678543618
 0.09171796107138079
 0.09134806335671229
 0.09048875946531719
 0.04832973229444604
 0.00607725632113932
 0.004669846317252394

In [19]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
        if s==Number_level
        s=1
        crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r=failure_penalty + normal_operation  ;
        return (sp=s, r=r)
       end
        r=0
     if a==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return (sp=s, r=r)
        end
    if a==2
        if s==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= normal_operation  ;
            return (sp=s, r=r)
        else
        s1=deepcopy(s)
        s=1
        r = maintenance_penalty 
         crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r+= normal_operation  ;
#            r*=(s1/10);
       return (sp=s, r=r)
        end
    end
    end

generative (generic function with 2 methods)

In [25]:
function ValueIteration(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,3))
#     print(Q)
    V= rand(Float64, (Number_level))*1000
    diff=0
    i=0
    
    

    
    p=[1 1 1 ]
    while i<=2000
#     while  i<=10000
    for s in 1:10
        Threads.@threads for a in 1:2
 
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:10 ))
    end
            V[s]=findmax(Q[s,1:2])[1]
        end
        i+=1
        end
    return Q,V
    end

ValueIteration (generic function with 3 methods)

In [33]:

# function decison(Q, s)
#    a_final=[]
#    if Number_level in s
# #         print("hi")
#         d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#         d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
#         min=-1e+10
#         opt=0
#         for (i,j) in enumerate(d)
#             if min < sum((j))
#                 min=sum((j))
#                 opt=i
#             end
#         end
#         return d1[opt]
#     end
#     q0=0.0
#     a_0=repeat(1:1,NumberUnits);
#     for i in 1:NumberUnits
#             q0+=Q[s[i],1]
#     end
#     d=(collect(Iterators.product((Q[s[i],2:3] for i in 1:NumberUnits)...)))
#     d1=(collect(Iterators.product((2:3 for i in 1:NumberUnits)...)))
# #     d[1]=a_0
# #     print("here")
# #     d1[1]=(1,1,1,1,1,1,1,1,1,1)
#     min=-1e+10
#     opt=0
#     for (i,j) in enumerate(d)
#         if i==1
#             if min < q0
#             min=q0
#             opt=i
#         end
#         elseif min < sum((j))
#             min=sum((j))
#             opt=i
#         end
#     end
#     if opt==1
#         return a_0
#     else
#         return d1[opt]
#     end
#     return opt
# end


function decison(Q, s,n=0.2)
   a_final=[]
   if true
#         print("hi")
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a], [1,2]))
            end
        return a_final
    end
end

decison (generic function with 2 methods)

In [37]:
cost=[ 
    [0 -1000 -200 -1000],
    [0 -1200 -200 -1000],
    [0 -1000 -200 -1400],
    [0 -1000 -200 -1600]
    
    ]

# print(Q)

for units in [60,50,40,30,20]
    for c in cost
   print(units)  
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty])   
     for lambda in 1:1:1
        global  lamb=lambda
    global NumberUnits=units
     Q,V=ValueIteration(s,T,.95,3000)
         println(isnan(Q[1,1]))
       if sum(isnan(Q[1,1]))>0.0
            continue
        end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
    r2=0.0
    s1=repeat(1:1,NumberUnits)
              
            
#     print(a)
    for i in 1:NumberUnits
#             print(generative(s[i],a[i],234))
         s1[i],r[i]=generative(s[i],a[i],234);
          r2+=r[i]
            end 
#     println(r2)
#     println(s)
    if (Number_level in s) || (2 in a)
            r2+=setup_cost
                    end
    s=s1;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,normal_operation,setup_cost,maintenance_penalty,failure_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end

60[0, -1000, -200, -1000]
false


[ Info:  started timer at: 2023-04-01T23:39:03.745


60
1
-90774.76304335413
3111.1234890108817


[ Info:           71.7338567s: 1 minute, 11 seconds, 733 milliseconds


-------------------------
60[0, -1200, -200, -1000]
false


...

[ Info:           16.9950852s: 16 seconds, 995 milliseconds


-------------------------
20[0, -1000, -200, -1600]
false


[ Info:  started timer at: 2023-04-01T23:53:51.256


20
1
-45142.678151559034
1989.1884683362812
-------------------------


[ Info:           12.9726529s: 12 seconds, 972 milliseconds


In [38]:
df

Row,Units,n,s,m,f,lambda,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any
1,20.0,0.0,-1000.0,-200.0,-1600.0,1.0,-45142.7,1989.19
2,20.0,0.0,-1000.0,-200.0,-1400.0,1.0,-44530.5,1803.66
3,20.0,0.0,-1200.0,-200.0,-1000.0,1.0,-46182.7,1959.95
4,20.0,0.0,-1000.0,-200.0,-1000.0,1.0,-42443.0,1899.95
5,30.0,0.0,-1000.0,-200.0,-1600.0,1.0,-58367.2,2420.21
6,30.0,0.0,-1000.0,-200.0,-1400.0,1.0,-57420.0,2154.37
7,30.0,0.0,-1200.0,-200.0,-1000.0,1.0,-58443.4,2266.77
8,30.0,0.0,-1000.0,-200.0,-1000.0,1.0,-54675.3,2238.81
9,40.0,0.0,-1000.0,-200.0,-1600.0,1.0,-71533.9,2796.67


In [39]:
using CSV
CSV.write("Independent with setup.csv", df)

"Independent with setup.csv"

In [40]:
 Q=ValueIteration(s,T,.95,3000)
for s in 1:Number_level
p=[exp(0.01*sum(Q[s,1])),exp(0.01*sum(Q[s,2])),exp(0.01*sum(Q[s,3]))]
    p./=sum(p)
    println(p)
end

LoadError: MethodError: no method matching getindex(::Tuple{Matrix{Float64}, Vector{Float64}}, ::Int64, ::Int64)
[0mClosest candidates are:
[0m  getindex(::Tuple, ::Int64) at tuple.jl:29
[0m  getindex(::Tuple, ::Integer) at tuple.jl:30
[0m  getindex(::Tuple, ::Real) at deprecated.jl:70
[0m  ...

In [41]:
using Plots

LoadError: ArgumentError: Package Plots not found in current path.
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.

In [42]:
for i in [3,5,10,15,20,25,30,40,50,60]
   display( plot(df[df.Units.==i,:].lambda,df[df.Units.==i,:].mean, xtick=df[df.Units.==i,:].lambda, show = true,title=string(i),label=false, xaxis=:log10,rotation=40))
    plot(show=true)
    println(df[df.Units.==i,:][findmax(df[df.Units.==i,:].mean)[2],:])

end

LoadError: UndefVarError: plot not defined

In [43]:
plot!()


LoadError: UndefVarError: plot! not defined